In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import h5py
import pandas as pd
import os
import numpy as np
import pylab as plt
from pesummary.gw.conversions.spins import spin_angles
import bilby
import sys
import pesummary
from pesummary.utils.samples_dict import SamplesDict
import sys

## Parameters

In [2]:
f_low = 10
f_ref = f_low

param_dict = {
    'mass_1': 38.64693598,
    'mass_2': 17.87341125,
    'luminosity_distance': 985.39919979,
    'theta_jn': 1.42311665,
    'ra': 1.8892089,
    'dec': 1.59878643,
    'psi': 2.01666645,
    'geocent_time': 0,
    'phase': 0,
    'a_1': 0.25482327,
    'a_2': 0.17168806,
    'tilt_1': 0.78275606,
    'tilt_2': 1.813414,
    'phi_12': 0.39800541,
    'phi_jl': 1.50715013,
    #'reference_frequency': f_ref
}
"""
param_dict = {
    'mass_1': 30,
    'mass_2': 20,
    'luminosity_distance': 500,
    'theta_jn': 0,
    'ra': np.pi/3,
    'dec': np.pi/4,
    'psi': np.pi/3,
    'geocent_time': 0,
    'phase': 0,
    'a_1': 0.0,
    'a_2': 0.0,
    'tilt_1': np.pi/4,
    'tilt_2': np.pi/6,
    'phi_12': np.pi/4,
    'phi_jl': np.pi/2,
    #'reference_frequency': f_ref
}
"""
approx = 'IMRPhenomXPHM'
delta_f = 1./100,
f_high = 4096

In [3]:
samples = SamplesDict({key:[val] for key, val in param_dict.items()})
samples.generate_all_posterior_samples(delta_f = delta_f, f_start = f_low, f_low=f_low, f_final=f_high, f_ref=f_ref)
samples

2025-09-16  00:19:56 PESummary WARNING : The redshift is being approximated using interpolation. Bear in mind that this does introduce a small error.
2025-09-16  00:19:56 PESummary INFO    : Averaging the final spin from the following fits: function bbh_final_spin_precessing_projected_UIB2016 at 0x79f70d7a27a0, function bbh_final_spin_precessing_projected_Healyetal at 0x79f70d7a25f0, function bbh_final_spin_precessing_HBR2016 at 0x79f70d7a2950
2025-09-16  00:19:56 PESummary INFO    : Averaging the peak luminosity from the following fits: function bbh_peak_luminosity_non_precessing_UIB2016 at 0x79f70d7a29e0, function bbh_peak_luminosity_non_precessing_Healyetal at 0x79f70d7a2a70
2025-09-16  00:19:56 PESummary INFO    : Averaging the final mass from the following fits: function bbh_final_mass_non_precessing_UIB2016 at 0x79f70d7a1f30, function bbh_final_mass_non_precessing_Healyetal at 0x79f70d7a0940


{'mass_1': Array([38.64693598]),
 'mass_2': Array([17.87341125]),
 'luminosity_distance': Array([985.39919979]),
 'theta_jn': Array([1.42311665]),
 'ra': Array([1.8892089]),
 'dec': Array([1.59878643]),
 'psi': Array([2.01666645]),
 'geocent_time': Array([0.]),
 'phase': Array([0.]),
 'a_1': Array([0.25482327]),
 'a_2': Array([0.17168806]),
 'tilt_1': Array([0.78275606]),
 'tilt_2': Array([1.813414]),
 'phi_12': Array([0.39800541]),
 'phi_jl': Array([1.50715013]),
 'redshift': Array([0.19537723]),
 'comoving_distance': Array([824.34162341]),
 'comoving_volume': Array([2.34644456e+09]),
 'mass_ratio': Array([0.46247939]),
 'inverted_mass_ratio': Array([2.16225853]),
 'total_mass': Array([56.52034723]),
 'chirp_mass': Array([22.55031149]),
 'symmetric_mass_ratio': Array([0.21622847]),
 'mass_1_source': Array([32.33032645]),
 'mass_2_source': Array([14.95210955]),
 'total_mass_source': Array([47.282436]),
 'chirp_mass_source': Array([18.86459854]),
 'iota': Array([1.34036117]),
 'spin_1x'

## PyCBC

In [4]:
from pycbc.waveform import get_fd_waveform
from pycbc.detector import Detector
from pycbc.filter.matchedfilter import sigmasq, sigma, FrequencySeries, TimeSeries
import pycbc

In [5]:
PyCBC_wf_gen_keys_map = {
    'mass_1':'mass1', 
    'mass_2':'mass2', 
    'luminosity_distance':'distance',
    'iota':'inclination',
    'geocent_time':'trigger_time',
    'phase':'coa_phase',
    'ra':'ra',
    'dec':'dec',
    'psi':'polarization',
    'spin_1x':'spin1x',
    'spin_1y':'spin1y',
    'spin_1z':'spin1z',
    'spin_2x':'spin2x',
    'spin_2y':'spin2y',
    'spin_2z':'spin2z',
    #'reference_frequency':'f_ref'
}

In [6]:
PyCBC_samples = {PyCBC_wf_gen_keys_map[key]:samples[key] for key in PyCBC_wf_gen_keys_map.keys()}
PyCBC_samples

{'mass1': Array([38.64693598]),
 'mass2': Array([17.87341125]),
 'distance': Array([985.39919979]),
 'inclination': Array([1.34036117]),
 'trigger_time': Array([0.]),
 'coa_phase': Array([0.]),
 'ra': Array([1.8892089]),
 'dec': Array([1.59878643]),
 'polarization': Array([2.01666645]),
 'spin1x': Array([-0.02321944]),
 'spin1y': Array([-0.17820422]),
 'spin1z': Array([0.18066271]),
 'spin2x': Array([0.04420253]),
 'spin2y': Array([-0.16069102]),
 'spin2z': Array([-0.0412471])}

In [7]:
def calc_PyCBC_snr(params_dict, f_low, f_high):
    hp, hc = get_fd_waveform(**params_dict)
    dets = ['L1', 'H1', 'I1']
    hf = {}
    snrs = {}
    psds = {}
    snr_sq_netw = 0
    for det in dets:
        fp, fc = Detector(det).antenna_pattern(params_dict['ra'], params_dict['dec'], params_dict['polarization'], params_dict['trigger_time'])
        hf[det] = hp*fp + hc*fc
        psds[det] = pycbc.psd.read.from_txt('../noise_curves/Asharp-asd.txt', len(hf[det]), hf[det].delta_f, f_low)
        snr_sq = sigmasq(hf[det], psd=psds[det], low_frequency_cutoff=f_low, high_frequency_cutoff=f_high)
        snrs[f'SNR_{det}'] = np.sqrt(snr_sq)
        snr_sq_netw += snr_sq
    snrs['SNR_network'] = np.sqrt(snr_sq_netw)
    return(hf, snrs, psds)

In [8]:
PyCBC_samples_wf_gen = {key: PyCBC_samples[key][0] for key in PyCBC_wf_gen_keys_map.values()}
PyCBC_samples_wf_gen.update(approximant=approx, f_lower=f_low, delta_f=delta_f[0], f_final=f_high, f_ref=f_ref)
PyCBC_samples_wf_gen

{'mass1': 38.64693598,
 'mass2': 17.87341125,
 'distance': 985.39919979,
 'inclination': 1.3403611696729574,
 'trigger_time': 0.0,
 'coa_phase': 0.0,
 'ra': 1.8892089,
 'dec': 1.59878643,
 'polarization': 2.01666645,
 'spin1x': -0.023219439502729415,
 'spin1y': -0.178204217619192,
 'spin1z': 0.1806627061277552,
 'spin2x': 0.044202532562217546,
 'spin2y': -0.16069101545104217,
 'spin2z': -0.04124710432215358,
 'approximant': 'IMRPhenomXPHM',
 'f_lower': 10,
 'delta_f': 0.01,
 'f_final': 4096,
 'f_ref': 10}

In [9]:
pycbc_hfs, pycbc_snrs, pycbc_psds = calc_PyCBC_snr(PyCBC_samples_wf_gen, f_low, f_high)

In [10]:
pycbc_snrs

{'SNR_L1': 16.49796390063354,
 'SNR_H1': 28.324489076649478,
 'SNR_I1': 13.823906951088633,
 'SNR_network': 35.57470868063714}

## Bilby

In [11]:
import bilby
bilby.core.utils.log.setup_logger(log_level="warning")

In [12]:
def calc_bilby_snr(injection_parameters):
    waveform_arguments = dict(
        waveform_approximant=approx,
        reference_frequency=f_ref,
        minimum_frequency=f_low
    )
    duration = 100
    # Create the waveform_generator using a LAL BinaryBlackHole source function
    waveform_generator = bilby.gw.WaveformGenerator(
        duration=duration,
        sampling_frequency=2*f_high,
        frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
        parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
        waveform_arguments=waveform_arguments
    )
    
    ifos = bilby.gw.detector.InterferometerList(["L1", "H1", "A1"])
    ifos.set_strain_data_from_zero_noise(
        sampling_frequency=2*f_high,
        duration=duration,
        start_time=injection_parameters["geocent_time"] - duration + 2,
    )
    
    for ifo in ifos:
        ifo.power_spectral_density = bilby.gw.detector.psd.PowerSpectralDensity(psd_file='../noise_curves/Asharp-psd.txt')
        ifo.minimum_frequency = f_low
    
    ifos.inject_signal(
        waveform_generator=waveform_generator, parameters=injection_parameters
    )

    snrdict_return = {}
    optsnrsquared = 0
    for ifo in ifos:
        det = ifo.name
        SNR = ifos.meta_data[det]["optimal_SNR"]
        snrdict_return[f'SNR_{det}'] = SNR
        optsnrsquared += SNR ** 2
    snrdict_return['SNR_network'] = np.sqrt(optsnrsquared)

    return snrdict_return, ifos

In [13]:
bilby_snr, ifos = calc_bilby_snr(param_dict)

In [14]:
bilby_snr, pycbc_snrs

({'SNR_L1': 16.441149253749053,
  'SNR_H1': 28.256434928232153,
  'SNR_A1': 13.785090777972817,
  'SNR_network': 35.479095695837884},
 {'SNR_L1': 16.49796390063354,
  'SNR_H1': 28.324489076649478,
  'SNR_I1': 13.823906951088633,
  'SNR_network': 35.57470868063714})

## GWBench

In [15]:
from gwbench import Network, injections_CBC_params_redshift

In [16]:
import gwbench

In [17]:
fm_params_str = 'Mc eta DL iota tc phic ra dec psi'
def run_analysis(inj_params, logging_level='WARNING', num_cores=2):
    f"""
    Parameters:
    -------------------------------------------------------------
    inj_params: dict
        dict with keys: Mc, eta, tc, phic, 
                        DL, iota, ra, dec, psi, 
                        chi1x, chi1y, chi1z, chi2x, chi2y, chi2z
    Returns:
    -----------------------
    FM: matrix
        Fisher matrix with parameter order as: {fm_params_str}
    cov: matrix
        Covariance matrix corresponding to the FM
    errors: dict
        Parameter errors
    """
    wf_model_name = 'lal_bbh'
    wf_other_var_dic = {'approximant':approx, 'fRef':f_ref}
    user_psds = {'A-Sharp':{'psd_file': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/Next_gen_detectability/scripts/next_gen_detect/noise_curves/Asharp-asd.txt', 
                            'is_asd': True},
                 'CE40':{'psd_file': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/Next_gen_detectability/scripts/next_gen_detect/noise_curves/CE40-asd.txt', 
                         'is_asd': True}, 
                 'CE20':{'psd_file': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/Next_gen_detectability/scripts/next_gen_detect/noise_curves/CE20-asd.txt', 
                         'is_asd': True}, 
                 'ET10-CoBA':{'psd_file': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/Next_gen_detectability/scripts/next_gen_detect/noise_curves/18213_ET10kmcolumns.txt', 
                              'is_asd': False}}
    user_locs = {'C1':{'longitude': -2.06175744538, 'latitude': 0.59637900541, 'arm_azimuth':0, 'which_arm':'y', 'shape':'L'}, 
                 'I1':{'longitude': 1.34444215058, 'latitude':0.34231676739, 'arm_azimuth':2.0527812170378947, #3.623591506466807, 
                       'which_arm':'y', 'shape':'L'}}
    conv_log = ('Mc', 'DL')
    ana_deriv_symbs_string = 'DL tc phic ra dec psi'
    network_spec = ['A-Sharp_L', 'A-Sharp_H', 'A-Sharp_I1']
    df = delta_f[0]
    f_arr = np.arange(f_low, f_high+df, df)
    net = Network(network_spec, logger_level=logging_level)
    net.set_net_vars(wf_model_name=wf_model_name, 
                     wf_other_var_dic=wf_other_var_dic,
                     f=f_arr, 
                     inj_params=inj_params,
                     deriv_symbs_string=fm_params_str,
                     ana_deriv_symbs_string=ana_deriv_symbs_string,
                     conv_log=conv_log, 
                     user_psds=user_psds,
                     user_locs=user_locs,
                     use_rot=False)
    net.calc_snrs()
    return(net)

In [18]:
gwbench_params_key = {
    'chirp_mass':'Mc',
    'symmetric_mass_ratio':'eta',
    'geocent_time':'tc',
    'phase':'phic',
    'luminosity_distance':'DL',
    'iota':'iota',
    'ra':'ra',
    'dec':'dec',
    'psi':'psi',
    'spin_1x':'chi1x',
    'spin_1y':'chi1y',
    'spin_1z':'chi1z',
    'spin_2x':'chi2x',
    'spin_2y':'chi2y',
    'spin_2z':'chi2z',
}

In [19]:
gwb_inj_params = {gwbench_params_key[key]:samples[key][0] for key in gwbench_params_key}
#gwb_inj_params.update(gmst0=bilby.gw.utils.greenwich_mean_sidereal_time(param_dict['geocent_time']))
gwb_inj_params

{'Mc': 22.55031149420461,
 'eta': 0.2162284690447004,
 'tc': 0.0,
 'phic': 0.0,
 'DL': 985.39919979,
 'iota': 1.3403611696729574,
 'ra': 1.8892089,
 'dec': 1.59878643,
 'psi': 2.01666645,
 'chi1x': -0.023219439502729415,
 'chi1y': -0.178204217619192,
 'chi1z': 0.1806627061277552,
 'chi2x': 0.044202532562217546,
 'chi2y': -0.16069101545104217,
 'chi2z': -0.04124710432215358}

In [20]:
net = run_analysis(gwb_inj_params)

In [21]:
for det in net.detectors:
    print(det.det_key, det.snr)
net.snr

A-Sharp_L 13.051645521406561
A-Sharp_H 23.739845628384415
A-Sharp_I1 11.297051553828874


29.35215656618508

In [22]:
pycbc_snrs

{'SNR_L1': 16.49796390063354,
 'SNR_H1': 28.324489076649478,
 'SNR_I1': 13.823906951088633,
 'SNR_network': 35.57470868063714}

In [23]:
bilby_snr

{'SNR_L1': 16.441149253749053,
 'SNR_H1': 28.256434928232153,
 'SNR_A1': 13.785090777972817,
 'SNR_network': 35.479095695837884}

In [40]:
gwfish_det_dict = {'L1':'L', 'H1':'H', 'I1':'IN'}

In [41]:
gwfish_input_data = pd.DataFrame({key:[val] for key, val in param_dict.items()})

In [42]:
gwfish_input_data

,mass_1,mass_2,luminosity_distance,theta_jn,ra,dec,psi,geocent_time,phase,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl
0,30,20,500,0.785398,1.047198,0.785398,1.047198,0,0.785398,0.0,0.0,0.785398,0.523599,0.785398,1.570796


In [62]:
# import GWFish.modules as gwf_mods
import pathlib
from GWFish.modules.detection import Detector
detectors = ['A_sharp_L', 'A_sharp_H', 'A_sharp_IN']
network = gwf_mods.detection.Network(detector_ids = detectors, detection_SNR = (0., 8.), config=pathlib.Path('detectors.yaml'))
gwfish_snr = gwf_mods.utilities.get_snr(gwfish_input_data, network, approx, f_ref=f_ref).rename(columns={f'A_sharp_{val}':f'SNR_{key}' for key, val in gwfish_det_dict.items()}).rename(columns={'network':'SNR_network'})

In [63]:
gwfish_snr

,SNR_L1,SNR_H1,SNR_I1,SNR_network
event_0,123.359161,108.477113,77.7113,181.724552


In [64]:
pycbc_snrs

{'SNR_L1': 123.31739868642305,
 'SNR_H1': 108.45946257957475,
 'SNR_I1': 77.71136958133104,
 'SNR_network': 181.6856978522047}

In [65]:
bilby_snr

{'SNR_L1': 123.389663014981,
 'SNR_H1': 108.5066750249657,
 'SNR_A1': 77.74804325309046,
 'SNR_network': 181.77861726179046}